In [1]:
import pandas as pd

# 데이터 경로
data_path = '/kaggle/input/porto-seguro-safe-driver-prediction/'

train = pd.read_csv(data_path + 'train.csv', index_col='id')
test = pd.read_csv(data_path + 'test.csv', index_col='id')
submission = pd.read_csv(data_path + 'sample_submission.csv', index_col='id')

In [2]:
all_data = pd.concat([train, test], ignore_index=True)
all_data = all_data.drop('target', axis=1) # 타깃값 제거

all_features = all_data.columns # 전체 피처
all_features

Index(['ps_ind_01', 'ps_ind_02_cat', 'ps_ind_03', 'ps_ind_04_cat',
       'ps_ind_05_cat', 'ps_ind_06_bin', 'ps_ind_07_bin', 'ps_ind_08_bin',
       'ps_ind_09_bin', 'ps_ind_10_bin', 'ps_ind_11_bin', 'ps_ind_12_bin',
       'ps_ind_13_bin', 'ps_ind_14', 'ps_ind_15', 'ps_ind_16_bin',
       'ps_ind_17_bin', 'ps_ind_18_bin', 'ps_reg_01', 'ps_reg_02', 'ps_reg_03',
       'ps_car_01_cat', 'ps_car_02_cat', 'ps_car_03_cat', 'ps_car_04_cat',
       'ps_car_05_cat', 'ps_car_06_cat', 'ps_car_07_cat', 'ps_car_08_cat',
       'ps_car_09_cat', 'ps_car_10_cat', 'ps_car_11_cat', 'ps_car_11',
       'ps_car_12', 'ps_car_13', 'ps_car_14', 'ps_car_15', 'ps_calc_01',
       'ps_calc_02', 'ps_calc_03', 'ps_calc_04', 'ps_calc_05', 'ps_calc_06',
       'ps_calc_07', 'ps_calc_08', 'ps_calc_09', 'ps_calc_10', 'ps_calc_11',
       'ps_calc_12', 'ps_calc_13', 'ps_calc_14', 'ps_calc_15_bin',
       'ps_calc_16_bin', 'ps_calc_17_bin', 'ps_calc_18_bin', 'ps_calc_19_bin',
       'ps_calc_20_bin'],
      dtype='obj

In [3]:
from sklearn.preprocessing import OneHotEncoder

# 명목형 피처 추출
cat_features = [feature for feature in all_features if 'cat' in feature] 

onehot_encoder = OneHotEncoder() # 원-핫 인코더 객체 생성
# 인코딩
encoded_cat_matrix = onehot_encoder.fit_transform(all_data[cat_features]) 

encoded_cat_matrix

<1488028x184 sparse matrix of type '<class 'numpy.float64'>'
	with 20832392 stored elements in Compressed Sparse Row format>

In [4]:
# 추가로 제거할 피처
drop_features = ['ps_ind_14', 'ps_ind_10_bin', 'ps_ind_11_bin', 
                 'ps_ind_12_bin', 'ps_ind_13_bin', 'ps_car_14']

# '1) 명목형 피처, 2) calc 분류의 피처, 3) 추가 제거할 피처'를 제외한 피처
remaining_features = [feature for feature in all_features 
                      if ('cat' not in feature and 
                          'calc' not in feature and 
                          feature not in drop_features)]

In [5]:
from scipy import sparse

all_data_sprs = sparse.hstack([sparse.csr_matrix(all_data[remaining_features]),
                               encoded_cat_matrix],
                              format='csr')

In [6]:
num_train = len(train) # 훈련 데이터 개수

# 훈련 데이터와 테스트 데이터 나누기
X = all_data_sprs[:num_train]
X_test = all_data_sprs[num_train:]

y = train['target'].values

In [7]:
import numpy as np

def eval_gini(y_true, y_pred):
    # 실제값과 예측값의 크기가 같은지 확인 (값이 다르면 오류 발생)
    assert y_true.shape == y_pred.shape

    n_samples = y_true.shape[0]                      # 데이터 개수
    L_mid = np.linspace(1 / n_samples, 1, n_samples) # 대각선 값

    # 1) 예측값에 대한 지니계수
    pred_order = y_true[y_pred.argsort()] # y_pred 크기순으로 y_true 값 정렬
    L_pred = np.cumsum(pred_order) / np.sum(pred_order) # 로렌츠 곡선
    G_pred = np.sum(L_mid - L_pred)       # 예측 값에 대한 지니계수

    # 2) 예측이 완벽할 때 지니계수
    true_order = y_true[y_true.argsort()] # y_true 크기순으로 y_true 값 정렬
    L_true = np.cumsum(true_order) / np.sum(true_order) # 로렌츠 곡선
    G_true = np.sum(L_mid - L_true)       # 예측이 완벽할 때 지니계수

    # 정규화된 지니계수
    return G_pred / G_true

In [8]:
# XGBoost용 지니함수 계산함수는 반환값이 2개(평가지표명, 평가점수)
# XGBoost용 gini() 함수
def gini(preds, dtrain):
    labels = dtrain.get_label()
    return 'gini', eval_gini(labels, preds)

In [9]:
import xgboost as xgb
from sklearn.model_selection import train_test_split

# 8:2 비율로 훈련 데이터, 검증 데이터 분리 (베이지안 최적화 수행용)
# X_train, X_valid, y_train, y_valid = train_test_split(X, y, 
#                                                       test_size=0.2, 
#                                                       random_state=0)
x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.2, 
                                                      random_state=0)

In [10]:
x_train.shape, x_test.shape

((476169, 201), (119043, 201))

In [11]:
import numpy as np
import pandas as pd
from keras.models import Sequential
from keras.layers import Dense, Conv1D, Flatten

In [12]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler(with_mean=False)
x_train = scaler.fit_transform(x_train)
x_test = scaler.transform(x_test)

In [17]:
display(x_train)
print(type(x_train))

<476169x201 sparse matrix of type '<class 'numpy.float64'>'
	with 12040605 stored elements in Compressed Sparse Row format>

<class 'scipy.sparse.csr.csr_matrix'>


In [19]:
x_train = x_train.toarray()

print(x_train.shape) #(476169, 201)
x_train_3d = x_train.reshape(x_train.shape[0], x_train.shape[1], 1)

# reshape 이후의 x_train shape 확인
print(x_train_3d.shape)  # (476169, 201, 1)

(476169, 201)
(476169, 201, 1)


In [27]:
from keras.layers import Dense, Conv1D, MaxPooling1D, Flatten

model = Sequential()
model.add(Conv1D(64, 3, activation='relu', input_shape=(x_train.shape[1], 1)))

model.add(Conv1D(64, 3, activation='relu'))

# MaxPooling1D layer
model.add(MaxPooling1D(pool_size=2))

# Conv1D layer 3
model.add(Conv1D(128, 3, activation='relu'))

# Conv1D layer 4
model.add(Conv1D(256, 3, activation='relu'))

# MaxPooling1D layer
model.add(MaxPooling1D(pool_size=2))

# Flatten layer
model.add(Flatten())

# Dense layer 1
model.add(Dense(64, activation='relu'))

# Dense layer 2
model.add(Dense(32, activation='relu'))

# 출력층(Dense layer 3)
model.add(Dense(1, activation='sigmoid'))

# 모델 컴파일
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])


In [30]:
x_train = np.expand_dims(x_train, axis=2)
model.fit(x_train, y_train, epochs=50, batch_size=32, validation_split=0.2)

# model.fit(x_train, y_train, epochs=50, batch_size=32, validation_split=0.2)

Epoch 1/50
11905/11905 [==============================] - 582s 49ms/step - loss: 0.1537 - accuracy: 0.9635 - val_loss: 0.1537 - val_accuracy: 0.9635
Epoch 2/50
11905/11905 [==============================] - 580s 49ms/step - loss: 0.1534 - accuracy: 0.9635 - val_loss: 0.1535 - val_accuracy: 0.9635
Epoch 3/50
11905/11905 [==============================] - 581s 49ms/step - loss: 0.1532 - accuracy: 0.9635 - val_loss: 0.1544 - val_accuracy: 0.9635
Epoch 4/50
11905/11905 [==============================] - 582s 49ms/step - loss: 0.1530 - accuracy: 0.9635 - val_loss: 0.1551 - val_accuracy: 0.9635
Epoch 5/50
11905/11905 [==============================] - 580s 49ms/step - loss: 0.1528 - accuracy: 0.9635 - val_loss: 0.1535 - val_accuracy: 0.9635
Epoch 6/50
11905/11905 [==============================] - 580s 49ms/step - loss: 0.1528 - accuracy: 0.9635 - val_loss: 0.1541 - val_accuracy: 0.9635
Epoch 7/50
11905/11905 [==============================] - 620s 52ms/step - loss: 0.1528 - accuracy: 0.9635

KeyboardInterrupt: 

In [33]:
x_test = x_test.toarray()

print(x_test.shape) #(476169, 201)
x_test_3d = x_test.reshape(x_test.shape[0], x_test.shape[1], 1)

x_test = np.expand_dims(x_test, axis=2)
y_test_preds = model.predict(x_test)

(119043, 201)
3721/3721 [==============================] - 55s 15ms/step


In [34]:
y_test_preds

array([[0.05182454],
       [0.03453731],
       [0.02037282],
       ...,
       [0.03757369],
       [0.05148397],
       [0.04524508]], dtype=float32)